## Subset and analyze Sentinel-2 imagey
* Fetches the Sentinel imagery layer from the Sentinel Living Atlas item:  
    * https://www.arcgis.com/home/item.html?id=fd61b9e0c69c4e14bebd50a9a968348c
* Creates a geometry (here, by geocoding)
* Filters the imagery for a specific time

In [11]:
#import packages
from arcgis import GIS
from arcgis.geocoding import geocode
from arcgis.raster import ImageryLayer, Raster
from arcgis.raster.functions import apply
from arcgis.geometry import Polygon, filters

import pandas as pd
from datetime import datetime

#instantitate the GIS object
gis = GIS('home')

In [2]:
#Fetch the imagery service and convert to a layer
s2_item = gis.content.get('fd61b9e0c69c4e14bebd50a9a968348c')
s2_images = s2_item.layers[0]

In [12]:
#Print the available raster functions
for rasterfunc in s2_images.properties.rasterFunctionInfos:
    print(rasterfunc.name)

Natural Color with DRA
Agriculture with DRA
Bathymetric with DRA
Color Infrared with DRA
Short-wave Infrared with DRA
Geology with DRA
Natural Color
Agriculture
Bathymetric
Geology
Short-wave Infrared
Color Infrared
NDVI Colormap
NDMI Colorized
Normalized Burn Ratio
Normalized Difference Built-Up Index (NDBI)
NDVI Raw
NDVI - VRE only Raw
NDVI - with VRE Raw
NDWI Raw
NDWI - with VRE Raw
None


In [20]:
#Get NWDI
ndwi_images = apply(s2_images, 'NDWI Raw')
color_infrared.extent = geocode('Redlands, CA', out_sr=ndwi_images.properties.spatialReference)[0].extent

AttributeError: 'dict' object has no attribute 'extent'

In [18]:
#Show 
m = gis.map("St. Louis, MO")
m.add_layer(ndwi_images)
m

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
#Get the spatial reference
s2_sr = s2_images.properties.spatialReference

In [4]:
#Create the geometry - we'll use St. Louis
stl = geocode(address = "St. Louis, MO", as_featureset=True , out_sr = s2_sr).sdf
stl_geom = stl.spatial.bbox
stl_filter = filters.intersects(stl_geom)

In [5]:
#Filter the Sentintal imagery layer for date and location
s2_image = s2_images.filter_by(
    geometry=stl_filter,
    time=[datetime(2024,11,10),datetime(2024,11,30)]
)

In [8]:
stl_geom.extent

(-10086295.713960564,
 4657223.948096092,
 -10040525.254383858,
 4687013.230869291)

In [9]:
s2_b1 = s2_image.select_bands(3,8)

13